In [0]:
!pip install kaggle


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 73.9 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "swatti"
os.environ['KAGGLE_KEY'] = "c18af5f0e868b40ee3eab947058ef2ff"
print("Kaggle credentials configured!")

Kaggle credentials configured!


In [0]:
spark.sql("""Create Schema if not exists workspace.ecommerce""")

DataFrame[]

In [0]:
spark.sql("""create volume if not exists workspace.ecommerce.ecommerce_data""")

DataFrame[]

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data 
kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store


Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors


100%|██████████| 4.29G/4.29G [00:34<00:00, 135MB/s]


In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data 
unzip -o ecommerce-behavior-data-from-multi-category-store
ls -lh


Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            
total 18G
-rwxrwxrwx 1 spark-578703f2-9c2b-4459-852f-2f nogroup 8.4G Jan 10 04:23 2019-Nov.csv
-rwxrwxrwx 1 spark-578703f2-9c2b-4459-852f-2f nogroup 5.3G Jan 10 04:25 2019-Oct.csv
-rwxrwxrwx 1 spark-578703f2-9c2b-4459-852f-2f nogroup 4.3G Jan 10 04:22 ecommerce-behavior-data-from-multi-category-store.zip


In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data 
rm -f ecommerce-behavior-data-from-multi-category-store.zip
ls -lh


total 0


In [0]:
%restart_python

In [0]:
df_n = spark.read.csv("/Volumes/workspace.ecommerce.ecommerce_data/2019-Nov.csv", header=True, inferSchema=True)

In [0]:
df_n = spark.read.format("csv").option("header", True).load(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv"
)
display(df_n.limit(5))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,null,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2


In [0]:
df = spark.read.csv("/Volumes/workspace.ecommerce.ecommerce_data/2019-Oct.csv", header=True, inferSchema=True)

In [0]:
df = spark.read.format("csv").option("header", True).load(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv"
)
display(df.limit(5))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,null,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,null,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [0]:
print(f"October 2019 - Total Events: {df.count():,}")
print("\n"+"="*60)
print("Schema:")
print("="*60)
df.printSchema()

October 2019 - Total Events: 42,448,764

Schema:
root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)



In [0]:
print("\n", "="*60)
print("sample data(first 5 rows): ")
df.show(5, truncate=False)


sample data(first 5 rows): 
+-----------------------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|event_time             |event_type|product_id|category_id        |category_code                      |brand   |price  |user_id  |user_session                        |
+-----------------------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|2019-10-01 00:00:00 UTC|view      |44600062  |2103807459595387724|NULL                               |shiseido|35.79  |541312140|72d76fde-8bb3-4e00-8c23-a032dfed738c|
|2019-10-01 00:00:00 UTC|view      |3900821   |2053013552326770905|appliances.environment.water_heater|aqua    |33.20  |554748717|9333dfbd-b87a-4708-9857-6336556b0fcc|
|2019-10-01 00:00:01 UTC|view      |17200506  |2053013559792632471|furniture.living_room.sofa         |NULL    |543.10 |519107250|5

In [0]:

#load your data
events = load_ecommerce_data("Oct")

#verify it's working
print(f"Ready to go! Loaded {events.count():,} events")
events.show(5)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-7993028875790939>, line 2
      1 #load your data
----> 2 events = load_ecommerce_data("Oct")
      4 #verify it's working
      5 print(f"Ready to go! Loaded {events.count():,} events")

TypeError: 'DataFrame' object is not callable